In [59]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

import sklearn
from sklearn.datasets import load_iris, load_wine
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.svm import SVC


In [60]:
from functools import wraps
import time


def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function {func.__name__}{args} {kwargs} Took {total_time:.4f} seconds')
        return result
    return timeit_wrapper


In [61]:
df = load_iris(as_frame=True)
df_wine = load_wine(as_frame=True)
predictors = df_wine.data
target = df_wine.target
target_name = df_wine.target_names


In [62]:
print(predictors.head(5))
print(target.head(5))
print(target_name)
print(df_wine.data.describe())

   alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0    14.23        1.71  2.43               15.6      127.0           2.80   
1    13.20        1.78  2.14               11.2      100.0           2.65   
2    13.16        2.36  2.67               18.6      101.0           2.80   
3    14.37        1.95  2.50               16.8      113.0           3.85   
4    13.24        2.59  2.87               21.0      118.0           2.80   

   flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
0        3.06                  0.28             2.29             5.64  1.04   
1        2.76                  0.26             1.28             4.38  1.05   
2        3.24                  0.30             2.81             5.68  1.03   
3        3.49                  0.24             2.18             7.80  0.86   
4        2.69                  0.39             1.82             4.32  1.04   

   od280/od315_of_diluted_wines  proline  
0                  

In [63]:
figure = px.histogram(target)
figure.show()

In [64]:
x_train, x_test, y_train, y_test = train_test_split(predictors, target, train_size=0.8, shuffle=True, random_state=271)
print(f'Размер для признаков обучающей выборки {x_train.shape}\n'
      f'Размер для признаков тестовой выборки {x_test.shape}\n'
      f'Размер для признаков обучающей выборки {y_train.shape}\n'
      f'Размер для признаков тестовой выборки {y_test.shape}\n')

Размер для признаков обучающей выборки (142, 13)
Размер для признаков тестовой выборки (36, 13)
Размер для признаков обучающей выборки (142,)
Размер для признаков тестовой выборки (36,)



In [65]:
@timeit
def logistical():
    m = LogisticRegression(random_state=271, solver='lbfgs', max_iter=10000)
    m.fit(x_train, y_train)
    return m

model = logistical()
y_predict = model.predict(x_test)
print(f'Предсказанные значения: \n{y_predict}\n'
      f'Исходные значения: \n{np.array(y_test)}')



Function logistical() {} Took 0.2904 seconds
Предсказанные значения: 
[2 1 0 1 1 2 1 1 1 1 1 0 1 0 1 2 1 1 1 2 2 2 1 2 2 0 0 1 2 2 2 1 1 1 0 1]
Исходные значения: 
[2 1 1 1 1 1 1 1 1 1 1 0 1 0 1 2 1 1 1 2 2 2 1 2 2 0 0 1 2 2 2 1 1 1 0 1]


In [66]:
# confusion_matrix = confusion_matrix(np.array(y_test), y_predict)
#
# cm_display = ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, )
#
# cm_display.plot()
# plt.show()
fig = px.imshow(confusion_matrix(np.array(y_test), y_predict), text_auto=True)
fig.update_layout(xaxis_title='Target', yaxis_title='Prediction')


In [67]:
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.83      1.00      0.91         5
           1       1.00      0.90      0.95        21
           2       0.91      1.00      0.95        10

    accuracy                           0.94        36
   macro avg       0.91      0.97      0.94        36
weighted avg       0.95      0.94      0.94        36



Support: количество наблюдений для каждого класса
Macro avg: среднее арифметическое показателя между классами
weighted avg: средневзвешенное значение рассчитывается путем произведения оценки показателя каждого класса на его количество наблюдений, последующее суммирование результата и деление результата на сумму наблюдений.


In [68]:
@timeit
def svc():
    param_kernel = ('linear', 'rbf', 'poly', 'sigmoid')
    parameters = {'kernel': param_kernel}
    m = SVC()
    grid = GridSearchCV(estimator = m, param_grid=parameters, cv=6)
    grid.fit(x_train, y_train)
    return grid

grid_search_svm = svc()


Function svc() {} Took 0.3114 seconds


In [69]:
best_model = grid_search_svm.best_estimator_
best_model.kernel

'linear'

In [70]:
svm_preds = best_model.predict(x_test)

print(f'Предсказанные значения: \n{svm_preds}\n'
      f'Исходные значения: \n{np.array(y_test)}')
print(classification_report(svm_preds, y_test))

Предсказанные значения: 
[2 1 0 1 1 2 1 1 1 1 1 0 1 0 1 2 1 1 1 2 2 2 1 2 2 0 0 0 2 2 2 1 1 1 0 1]
Исходные значения: 
[2 1 1 1 1 1 1 1 1 1 1 0 1 0 1 2 1 1 1 2 2 2 1 2 2 0 0 1 2 2 2 1 1 1 0 1]
              precision    recall  f1-score   support

           0       1.00      0.71      0.83         7
           1       0.86      1.00      0.92        18
           2       1.00      0.91      0.95        11

    accuracy                           0.92        36
   macro avg       0.95      0.87      0.90        36
weighted avg       0.93      0.92      0.91        36



In [71]:
fig = px.imshow(confusion_matrix(y_test, svm_preds), text_auto=True)
fig.update_layout(xaxis_title='Target', yaxis_title='Prediction')


In [72]:
@timeit
def classifier():
    number_of_neighbors = np.array([3, 10, 25])
    model_KNN = KNeighborsClassifier()
    params = {'n_neighbors': number_of_neighbors}

    grid_search = GridSearchCV(estimator=model_KNN, param_grid=params, cv=6)
    grid_search.fit(x_train, y_train)

    return grid_search

grid_search = classifier()

Function classifier() {} Took 0.0600 seconds


In [73]:
grid_search.best_estimator_

KNeighborsClassifier(n_neighbors=3)

In [74]:
knn_preds = grid_search.predict(x_test)
print(f'Предсказанные значения: \n{knn_preds}\n'
      f'Исходные значения: \n{np.array(y_test)}')
print(classification_report(knn_preds, y_test))

Предсказанные значения: 
[2 1 0 0 1 2 1 2 1 1 1 0 1 0 1 1 1 1 1 2 1 2 1 2 0 0 1 1 1 1 2 1 1 2 0 1]
Исходные значения: 
[2 1 1 1 1 1 1 1 1 1 1 0 1 0 1 2 1 1 1 2 2 2 1 2 2 0 0 1 2 2 2 1 1 1 0 1]
              precision    recall  f1-score   support

           0       0.80      0.57      0.67         7
           1       0.76      0.76      0.76        21
           2       0.50      0.62      0.56         8

    accuracy                           0.69        36
   macro avg       0.69      0.65      0.66        36
weighted avg       0.71      0.69      0.70        36



In [75]:
fig = px.imshow(confusion_matrix(y_test, knn_preds), text_auto=True)
fig.update_layout(xaxis_title='Target', yaxis_title='Prediction')
